On peut interragir avec une base sql de nombreuses manières possibles:
1. Grace à une interface graphique 
     - Phpmyadmin pour le CRUD en mysql
     - Vscode Viewer mais que pour le read
 2. Dans le terminal: nécessaire quand on travaille dans des contenaires et souvent utile dans le cloud.
 3. En executant du code sql à l'aide d'un toolkit: nécessite soit de coder du sql dans un fichier à part, ou dans un fichier python mais sans aide de l'IDE. a l'avatage de s'appuyer sur du code SQL qui est universel
 4. Avec un toolkit en mode requete: permet d'automatiser certaines choses mais de manière aussi poussée qu'avec un ORM et sans être universel comme SQL
      - sqlite
      - sqlalchemy 
2. Avec un ORM: permet d'exploiter la puissance de l'orienté objet (facilité de lecture et factorisation importante)
     - sqlalchemy.

# I Utilisation du terminal

## 1. création des tables

sqlite> .mode column  
sqlite> .header on  
sqlite> .read c:/sqlite/commands.txt  

## 2. Importation des données

A partir d'un fichier csv pays.csv.

On se connecte à notre base en tapant **`Sqlite3`** dans le terminal puis:

- .open McDo.db
- .mode csv
- .import pays.csv Pays 

# II En utilisant du sql dans python

## 1. création des tables

In [ ]:
from sqlalchemy import text, declarative_base, create_engine

engine = create_engine("sqlite:///burger.db")
Base = declarative_base()
Base.metadata.create_all(bind=engine)

sep = '\n\n'
with open("sqlite_table_creation.txt", 'r') as sql_file:
    insert_request_string = sql_file.read()

insert_request_List = insert_request_string.split(sep)

with engine.connect() as con:
    for insert_request in insert_request_List :
        statement = text(insert_request)
        con.execute(statement)

## 2. importation des données

In [ ]:
from pandas import Dataframe

df_pays = Dataframe()

df_pays.to_sql(name='Pays', con=engine, if_exists = 'append', index=False)

## 3. Queries

In [ ]:
connection = engine.connect()

output = connection.execute("""
    SELECT code_postal, count(id_employe) AS nb_employe 
    FROM Employe 
    GROUP BY code_postal; 
""")



# III En utilisant Sqlalchemy en mode Core

## 1. création des tables

In [12]:
from sqlalchemy import Table, Column, MetaData, ForeignKey, create_engine, String, Integer

In [ ]:
# On établit une connexion
engine = create_engine("sqlite:///burger.db")
connection = engine.connect()

In [ ]:
# On instancie un objet metadata vide
metadata = MetaData()

# On ajoute aux métadata différentes tables: 
pays_table = Table('Pays', metadata,
              Column('pays', String(), primary_key=True)
              )

### Gestion des restaurants
restaurant_table = Table('Restaurant', metadata,
              Column('code_postal', String(), primary_key=True),
              Column('pays', String(), ForeignKey("Pays.pays"), nullable=False),
              Column('capacité', Integer(), default=0),
              Column('espace_enfant', Integer(), default=False),
pays_table = Table('Pays', metadata,
              Column('pays', String(), primary_key=True)
              )

# On crée les tables ajoutées aux métadatas dans notre BDD
metadata.create_all(engine) 

## 2. importation des données

In [ ]:
# Set up connections between sqlalchemy and postgres dbapi
engine = create_engine("sqlite:///burger.db")
connection = engine.connect()

# On instancie un objet Metadata vide
metadata = MetaData()

# Avec reflect, on ajoute à l'instance métédata les données de notre BDD
with engine.connect() as conn:
    metadata.reflect(conn)

# De manière optionnelle, on importe les packages permettant de générer des donées

from faker import Faker

faker = Faker()
faker.random.seed(42)

In [ ]:
pays_table = metadata.tables["Pays"]

# With permet d'ouvrir une connexion unique pour l'ensemble des requetes, si un échoue, aucune n'est lancée
with engine.begin() as conn:
    for _ in range(5):
        insert_stmt = pays_table.insert().values(
        pays = faker.country()
        )
        conn.execute(insert_stmt)

In [ ]:
from sqlalchemy import select
import random
restaurant_table = metadata.tables["Restaurant"]

# à l'aide de notre connexion, on va chercher les clés primaires de la table restaurant.
pays_pk = connection.execute(select([pays_table.c.pays])).fetchall()

with engine.begin() as conn:
    for _ in range(50):
      insert_stmt = restaurant_table.insert().values(
        code_postal = faker.postalcode(),
        pays = random.choice(pays_pk)[0],
        capacité = faker.pyint(0,500),
        espace_enfant = faker.pyint(0,1),
        service_rapide = faker.pyint(0,1),
        accessibilité = faker.pyint(0,1),
        parking = faker.pyint(0,200),
      )
      conn.execute(insert_stmt)

## 3. Queries

In [ ]:
engine = create_engine("sqlite:///burger.db")
connection = engine.connect()

# Instantiate metadata class
metadata = MetaData()

# Reflect metadata/schema from existing sqlite database
with engine.connect() as conn:
    metadata.reflect(conn)

In [ ]:
from sqlalchemy import func

In [ ]:
employe = metadata.tables["Employe"]
restaurant = metadata.tables["Restaurant"]

employe_list = connection.execute(
    select( [func.count(employe.c.id_employe), func.substring(employe.c.code_postal,0,3),employe.c.pays])
    .group_by(restaurant.c.pays,  func.substring(restaurant.c.code_postal,0,3))
    ).fetchall()
employe_list

voir [doc](https://docs.sqlalchemy.org/en/14/orm/queryguide.html)

# IV En utilisant sqlalchemy en mode ORM

In [ ]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Boolean, SmallInteger, CheckConstraint


## 1. création des tables

In [ ]:
engine = create_engine('sqlite:///db.sqlite')
Base = declarative_base()


class Pays(Base):
    __tablename__ = "Pays"

    pays = Column(String, primary_key=True)



class Restaurant(Base):
    __tablename__ = "Restaurant"

    code_postal = Column(String, primary_key=True)
    departement = Column(String)
    pays = Column(String, ForeignKey('Pays.pays'))
    capacite = Column(Integer, default=0)
    espace_enfant = Column(SmallInteger, default=0)
    service_rapide = Column(SmallInteger, default=0)
    accessibilite = Column(SmallInteger, default=0)
    parking = Column(SmallInteger, default=0)


Base.metadata.create_all(engine)

## 2. Importation des données

In [ ]:
from sqlalchemy.orm import sessionmaker
from faker import Faker

Session = sessionmaker(bind=engine)
session = Session()

faker = Faker()


for _ in range(10):
    session.add(Pays(pays=faker.country()))

session.commit()

## 3. Queries

In [ ]:
Pays.query.get("France")

# ou 

session.query(Pays).filter(Pays.continent = "America").all()

In [ ]:
pays = Pays(pays="France")
pays.get_all_restaurant()

voir [doc](https://docs.sqlalchemy.org/en/14/orm/query.html)

## 4. Drop Delete

### Supprimer une table

In [ ]:
from sqlite_init import Gare
from sqlalchemy import create_engine

engine = create_engine('sqlite:///db.sqlite')
Gare.__table__.drop(engine)

### Nettoyer les lignes d'une table

In [ ]:
session.query(Gare).delete()
session.commit()